# Two-way Fixed Effects

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import gpboost as gpb
from patsy import dmatrix

In [2]:
path = '../../../../data/processed/df_filtered.xlsx'
df = pd.read_excel(path)
df['log_pop'] = np.log(df['populations'])
df.dropna(subset=['log_pop'], inplace=True)
df.head()

,island_ids,years,region_codes,prefecture_codes,populations,treatment_group,treatment_group_mainland,year_bridges_opened,after_treated,year_connect_mainland,...,area_km2,distance_m,elementary_school,junior_high_school,high_school,hospital,population_change_rate,control_group,time_since_intervention,log_pop
5,67,1995,32202,32,426.0,1,1,1999,0,1999,...,0.28,615.0,NaN,NaN,NaN,NaN,NaN,0,-4,6.054439
6,67,2000,32202,32,324.0,1,1,1999,1,1999,...,0.28,615.0,NaN,NaN,NaN,NaN,-23.943662,0,1,5.780744
7,67,2005,32202,32,302.0,1,1,1999,1,1999,...,0.28,615.0,NaN,NaN,NaN,NaN,-6.790123,0,6,5.710427
8,67,2010,32202,32,265.0,1,1,1999,1,1999,...,0.28,615.0,NaN,NaN,NaN,NaN,-12.251656,0,11,5.579730
9,67,2015,32202,32,197.0,1,1,1999,1,1999,...,0.28,615.0,NaN,NaN,NaN,NaN,-25.660377,0,16,5.283204


In [20]:
y = df['log_pop']
group = df[['island_ids', 'years']]
X = dmatrix('after_treated', data=df, return_type='dataframe')

gp_model = gpb.GPModel(group_data=group)
gp_model.fit(y, X, params={'std_dev': True})
gp_model.summary()

Model summary:
 Log-lik    AIC    BIC
 -310.78 631.56 658.74
Nb. observations: 1697
Nb. groups: 94 (island_ids), 30 (years)
-----------------------------------------------------
Covariance parameters (random effects):
            Param.  Std. dev.
Error_term  0.0524     0.0019
island_ids  3.9112     0.5720
years       0.2233     0.0600
-----------------------------------------------------
Linear regression coefficients (fixed effects):
               Param.  Std. dev.  z value  P(>|z|)
Intercept      5.7641     0.2222  25.9440      0.0
after_treated  0.2743     0.0283   9.6968      0.0
